This notebook reduces the size of our network for visualisation purposes.

In [12]:
import os
import re

import numpy as np
import pandas as pd

from helper import display_df_with_bokeh

pd.set_option('display.width', 400)

In [13]:
DATASETS_FOLDER = "datasets"

In [14]:
# Opening original network with modularity
with open(f"gephi/citations/nodes_with_all_network_stats.csv", 'r') as f:
    gephi_df = pd.read_csv(f, header=0)

In [21]:
# Calculate members per group
gephi_df.groupby('modularity_class').count().sort_values('Id', ascending = False).head(10)

,Id,Label,timeset,indegree,outdegree,Degree,weighted indegree,weighted outdegree,Weighted Degree,Authority,Hub,componentnumber,strongcompnum,pageranks,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,clustering,eigencentrality
modularity_class,,,,,,,,,,,,,,,,,,,,
0,2447,2447,0,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447
23,2270,2270,0,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
4,2081,2081,0,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081
1,2030,2030,0,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030
2,1922,1922,0,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922
30,1632,1632,0,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632
13,1494,1494,0,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494
22,1485,1485,0,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485
5,1481,1481,0,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481


In [25]:
# Calculate citations per group
gephi_df.groupby('modularity_class').sum().sort_values('indegree', ascending = False).head(10)

,Id,Label,timeset,indegree,outdegree,Degree,weighted indegree,weighted outdegree,Weighted Degree,Authority,Hub,componentnumber,strongcompnum,pageranks,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,clustering,eigencentrality
modularity_class,,,,,,,,,,,,,,,,,,,,
0,16243332655,16243332655,0.0,51168,46592,97760,51168,46592,97760,4.113399,7.754813,0,27529577,0.138198,26893,608.321344,687.134898,6.560148e+07,374.868718,24.296273
1,10126469892,10126469892,0.0,42893,40999,83892,42893,40999,83892,8.260464,24.562775,0,28250352,0.071957,25495,446.075048,501.008393,8.674125e+07,344.407388,20.198210
2,12613898971,12613898971,0.0,37960,34414,72374,37960,34414,72374,4.131561,9.170672,0,23172782,0.099461,23545,434.483930,489.665774,6.583425e+07,291.128950,18.605380
4,15003327824,15003327824,0.0,35222,33699,68921,35222,33699,68921,2.202700,4.792904,0,24544041,0.093857,21991,504.071322,569.809792,4.647956e+07,344.306724,14.186341
13,11772199866,11772199866,0.0,27150,24497,51647,27150,24497,51647,2.714990,4.861421,0,14703265,0.118747,14407,409.342895,465.747552,3.358218e+07,211.193606,14.337181
30,4551969809,4551969809,0.0,24667,28128,52795,24667,28128,52795,1.399801,4.496187,0,26910244,0.045039,19840,392.514474,429.713566,4.519072e+07,263.444249,9.449359
3,2516415513,2516415513,0.0,19007,21245,40252,19007,21245,40252,0.899520,2.280505,0,14664287,0.027903,11011,208.396193,232.004825,2.658279e+07,154.285013,7.729578
25,2233764580,2233764580,0.0,17959,20584,38543,17959,20584,38543,1.356976,4.723676,0,20568629,0.032607,16525,259.831837,288.178296,2.641750e+07,198.749061,5.922142
23,14388144878,14388144878,0.0,11909,11959,23868,11909,11959,23868,0.075140,0.242091,0,33914882,0.059079,18882,714.953160,781.185200,1.872865e+07,373.462228,3.137336


In [17]:
# Calculate records per group
modularity_df = gephi_df.groupby('modularity_class').count().sort_values('Id', ascending = False)

# Only taking in consideration where groups have more than 10 nodes
modularity_mt_nodes_df = modularity_df[modularity_df.Id >= 10]

# display_df_with_bokeh(modularity_mt_nodes_df, include_index=True)
modularity_mt_nodes_df

,Id,Label,timeset,indegree,outdegree,Degree,weighted indegree,weighted outdegree,Weighted Degree,Authority,Hub,componentnumber,strongcompnum,pageranks,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,clustering,eigencentrality
modularity_class,,,,,,,,,,,,,,,,,,,,
0,2447,2447,0,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447,2447
23,2270,2270,0,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
4,2081,2081,0,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081
1,2030,2030,0,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030
2,1922,1922,0,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922,1922
30,1632,1632,0,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632,1632
13,1494,1494,0,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494,1494
22,1485,1485,0,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485,1485
5,1481,1481,0,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481


In [18]:
print("Total nodes that form part of communities with more than 10 members: ", modularity_mt_nodes_df.sum()['Id'])


Total nodes that form part of communities with more than 10 members:  27379


In [19]:
print("Total nodes that form part of communities with less than 10 members: ", modularity_df[modularity_df.Id < 10].sum()['Id'])


Total nodes that form part of communities with less than 10 members:  363


In [7]:
modularity_df[modularity_df.Id < 10]

,Id,Label,timeset,indegree,outdegree,Degree,weighted indegree,weighted outdegree,Weighted Degree,Authority,Hub,componentnumber,strongcompnum,pageranks,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,clustering,eigencentrality
modularity_class,,,,,,,,,,,,,,,,,,,,
102,8,8,0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
11,7,7,0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
24,6,6,0,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
109,6,6,0,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
90,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
36,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
95,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
153,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
72,5,5,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [ ]:
gephi_df[gephi_df['modularity_class'] == 2]

In [ ]:
total_nodes = modularity_mt_nodes_df.sum()['Id']
total_nodes

In [ ]:
# DF will contain the most important nodes of each modularity class
nodes_per_modularity_class_df = None

In [ ]:
for row in modularity_mt_nodes_df.iterrows():
    
    modularity = row[0]
    
    total_nodes_in_class = modularity_mt_nodes_df.loc[modularity]['Id']
    proportion_of_network = (total_nodes_in_class / total_nodes)
    subset_of_nodes = int(round(proportion_of_network * total_nodes_in_class)) + 1
    
    print(f"Modularity class: {modularity} - Total nodes in class: {total_nodes_in_class} - Proportion of final network: {proportion_of_network} - New total {subset_of_nodes}")
    
    aux = gephi_df[gephi_df.modularity_class == modularity].sort_values('Degree', ascending=False)[:subset_of_nodes]
    
    if nodes_per_modularity_class_df is None:
        nodes_per_modularity_class_df = aux
    else:
        nodes_per_modularity_class_df = nodes_per_modularity_class_df.append(aux, ignore_index=True)

In [ ]:
display_df_with_bokeh(nodes_per_modularity_class_df)
print("Most important nodes of our network: ", nodes_per_modularity_class_df.shape[0])

In [ ]:
with open(f"{DATASETS_FOLDER}/cit-HepTh.txt", 'r') as f:
    hepth_df = pd.read_csv(f,sep='\t',skiprows=(0,1,2))
    
# Rename columns
hepth_df.columns = ['FromNodeId', 'ToNodeId']

In [ ]:
display_df_with_bokeh(hepth_df)

In [ ]:
nodes_with_modularity_df = nodes_per_modularity_class_df[['Id', 'modularity_class']]
display_df_with_bokeh(nodes_with_modularity_df)

In [ ]:
# Join the original network with our most important nodes with modularity
hepth_df_with_modularity = hepth_df.merge(nodes_with_modularity_df, how="outer", left_on = 'FromNodeId', right_on = 'Id')

hepth_df_with_modularity.columns = ['FromNodeId', 'ToNodeId', 'Id', 'modularity_from']
hepth_df_with_modularity.drop(['Id'], axis=1, inplace=True)

hepth_df_with_modularity = hepth_df_with_modularity.merge(nodes_with_modularity_df, how="outer", left_on = 'ToNodeId', right_on = 'Id')

hepth_df_with_modularity.columns = ['FromNodeId', 'ToNodeId', 'modularity_from', 'Id', 'modularity_to']
hepth_df_with_modularity.drop(['Id'], axis=1, inplace=True)

In [ ]:
# Remove all edges that have a node that was not found in nodes_with_modularity_df
hepth_df_with_modularity.dropna(inplace=True)
hepth_df_with_modularity = hepth_df_with_modularity.astype(int)

In [ ]:
display_df_with_bokeh(hepth_df_with_modularity)
print(hepth_df_with_modularity.shape)

In [ ]:
# Save our new reduced network with most important nodes
hepth_df_with_modularity.to_csv("gephi/reduced_size_network.csv")